<a href="https://colab.research.google.com/github/foxtrotmike/CS909/blob/master/affine_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np

class AffineTransformation:
    def __init__(self):
        self.transformation_matrix = None

    def compute_transformation_matrix(self, X, Y):
        # Recovers affine transformation to map X to Y
        # Ensure X and Y are numpy arrays
        X = np.array(X)
        Y = np.array(Y)

        # Calculate the centroids of both sets
        centroid_X = np.mean(X, axis=0)
        centroid_Y = np.mean(Y, axis=0)

        # Center the points
        X_centered = X - centroid_X
        Y_centered = Y - centroid_Y

        # Calculate the scaling factor
        norm_X = np.linalg.norm(X_centered)
        norm_Y = np.linalg.norm(Y_centered)
        scale = norm_Y / norm_X

        # Scale the centered points
        X_scaled = X_centered * scale

        # Calculate the rotation using Singular Value Decomposition (SVD)
        U, S, Vt = np.linalg.svd(np.dot(Y_centered.T, X_scaled))
        R = np.dot(U, Vt)

        # Combine the transformations
        self.transformation_matrix = np.eye(3)
        self.transformation_matrix[:2, :2] = scale * R
        self.transformation_matrix[:2, 2] = centroid_Y - scale * np.dot(R, centroid_X)

    def transform_points(self, points):
        if self.transformation_matrix is None:
            raise ValueError("Transformation matrix has not been computed yet.")

        # Ensure points are numpy arrays and homogeneous coordinates
        points = np.array(points)
        ones = np.ones((points.shape[0], 1))
        points_homogeneous = np.hstack((points, ones))

        # Apply the transformation
        transformed_points_homogeneous = np.dot(self.transformation_matrix, points_homogeneous.T).T

        # Convert back to Cartesian coordinates
        transformed_points = transformed_points_homogeneous[:, :2]
        return transformed_points
if __name__=='__main__':
  # Generate test points
  np.random.seed(42)
  points_X = np.random.rand(10, 2) * 10  # 10 random points in 2D

  # Define a known transformation matrix (scaling, rotation, translation)
  scale = 1.5
  theta = np.radians(30)
  translation = np.array([2, 3])

  R = np.array([
      [np.cos(theta), -np.sin(theta)],
      [np.sin(theta), np.cos(theta)]
  ])

  transformation_matrix = np.eye(3)
  transformation_matrix[:2, :2] = scale * R
  transformation_matrix[:2, 2] = translation

  # Apply the known transformation to points_X
  affine_transformation = AffineTransformation()
  affine_transformation.transformation_matrix = transformation_matrix
  points_Y = affine_transformation.transform_points(points_X)

  # Recover the transformation matrix using the compute_transformation_matrix method
  recovered_affine_transformation = AffineTransformation()
  recovered_affine_transformation.compute_transformation_matrix(points_X, points_Y)

  # Print results
  print("Original Transformation Matrix:\n", transformation_matrix)
  print("Recovered Transformation Matrix:\n", recovered_affine_transformation.transformation_matrix)

  # Check if the recovered transformation matrix is close to the original one
  print("Transformation matrices are close:", np.allclose(transformation_matrix, recovered_affine_transformation.transformation_matrix))


Original Transformation Matrix:
 [[-1.29903811  0.75        2.        ]
 [-0.75       -1.29903811  3.        ]
 [ 0.          0.          1.        ]]
Recovered Transformation Matrix:
 [[-1.29903811  0.75        2.        ]
 [-0.75       -1.29903811  3.        ]
 [ 0.          0.          1.        ]]
Transformation matrices are close: True


In [11]:
recovered_affine_transformation.transform_points(points_X)

array([[  4.26493843, -12.15919201],
       [ -3.0189416 , -10.2667564 ],
       [  1.14321731,  -0.19656807],
       [  7.74179284,  -8.68758529],
       [ -0.49816873, -10.70649519],
       [  9.00692347,  -9.75388228],
       [ -7.22120378,  -6.00168577],
       [  1.01355821,  -0.74618172],
       [  1.98345057,  -6.09860283],
       [ -1.42691184,  -4.02276515]])

In [10]:
points_Y

array([[  4.26493843, -12.15919201],
       [ -3.0189416 , -10.2667564 ],
       [  1.14321731,  -0.19656807],
       [  7.74179284,  -8.68758529],
       [ -0.49816873, -10.70649519],
       [  9.00692347,  -9.75388228],
       [ -7.22120378,  -6.00168577],
       [  1.01355821,  -0.74618172],
       [  1.98345057,  -6.09860283],
       [ -1.42691184,  -4.02276515]])